## Get feature values

Retrieve feature values by entity ID

In [123]:
import json

import numpy as np
from google.cloud import bigquery
from google.cloud.aiplatform_v1 import FeatureOnlineStoreServiceClient
from google.cloud.aiplatform_v1.types import (
    feature_online_store_service as feature_online_store_service_pb2,
)
from google.cloud.aiplatform_v1beta1.types import NearestNeighborQuery

PROJECT_ID = "vertex-feature-store"
PROJECT_NUMBER = "208794590551"
REGION = "europe-west2"
FEATUREONLINESTORE_NAME = "example_online_store"
FEATUREVIEW_BASE = f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATUREONLINESTORE_NAME}/featureViews/"

In [9]:
def get_feature_values(
    data_client: FeatureOnlineStoreServiceClient, feature_view: str, entitiy_id: str
):
    return data_client.fetch_feature_values(
        request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
            feature_view=feature_view,
            data_key=feature_online_store_service_pb2.FeatureViewDataKey(
                key=entitiy_id
            ),
        )
    )


data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

In [10]:
feature_view_name = "user_featureview"
feature_view = f"{FEATUREVIEW_BASE}{feature_view_name}"
entity_id = "1"

response = get_feature_values(data_client, feature_view, entity_id)
print(response)

key_values {
  features {
    value {
      string_value: "user1"
    }
    name: "username"
  }
  features {
    value {
      string_value: "user1@gmail.com"
    }
    name: "email"
  }
  features {
    value {
      int64_value: 25
    }
    name: "age"
  }
  features {
    value {
      string_value: "M"
    }
    name: "gender"
  }
  features {
    value {
      int64_value: 1707764360639560
    }
    name: "feature_timestamp"
  }
}



In [131]:
feature_view_name = "user_rating_featureview"
feature_view = f"{FEATUREVIEW_BASE}{feature_view_name}"
entity_id = "1"

response = get_feature_values(data_client, feature_view, entity_id)
print(response)

NotFound: 404 Requested entity was not found.

### Search with embedding nearest neighbor


[Retrieve the public endpoint domain name for the online store (docs 2024-02-11)](https://cloud.google.com/vertex-ai/docs/featurestore/latest/embeddings-search#retrieve-publicendpoint)

When you create and configure an online store instance for embedding management, Vertex AI Feature Store generates a public endpoint domain name for the online store. Before you can start searching for nearest neighbors from a feature view in the online store, you must retrieve the public endpoint domain name from the online store details.


In [12]:
! curl -X GET \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    "https://europe-west2-aiplatform.googleapis.com/v1beta1/projects/208794590551/locations/europe-west2/featureOnlineStores/example_online_store"

{
  "name": "projects/208794590551/locations/europe-west2/featureOnlineStores/example_online_store",
  "createTime": "2024-02-12T18:52:35.549370Z",
  "updateTime": "2024-02-12T18:52:39.148180Z",
  "etag": "AMEw9yPm833KTm592J_5gRBu5zQPJhzLjXxcoLpR2Lttq4xCnJ49M8UqsBw8aROeH_0=",
  "state": "STABLE",
  "bigtable": {
    "autoScaling": {
      "minNodeCount": 1,
      "maxNodeCount": 1,
      "cpuUtilizationTarget": 80
    }
  },
  "dedicatedServingEndpoint": {
    "publicEndpointDomainName": "1734071774.europe-west2-1012421088783.vdb.vertexai.goog"
  },
  "embeddingManagement": {
    "enabled": true
  }
}


In [122]:
# Use a movie for which we want to find similar movies
bq_client = bigquery.Client(project=PROJECT_ID)

dataset = "example_dataset"
table = "movie_with_embedding_view"
query = f"SELECT * FROM {PROJECT_ID}.{dataset}.{table} WHERE entity_id = '1'"

df = bq_client.query(query).to_dataframe()
display(df)

,entity_id,movie_name,genre,genre_code,language,running_time,embedding,feature_timestamp
0,1,The Shawshank Redemption,Drama,5,English,142,"[-0.055181887, 0.037215218, 0.007869319, 0.034...",2024-02-12 20:07:29.831959+00:00


Search by embedding


In [124]:
query_embedding = df["embedding"].values[0]
json_data = json.dumps(
    {
        "query": {"embedding": {"value": np.array(query_embedding).tolist()}},
        "neighbour_count": 3,
        "return_full_entity": True,
    }
)

In [125]:
! curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d '{json_data}' \
    "https://1734071774.europe-west2-1012421088783.vdb.vertexai.goog/v1beta1/projects/208794590551/locations/europe-west2/featureOnlineStores/example_online_store/featureViews/movie_emb_featureview:searchNearestEntities" \

{"nearestNeighbors":{"neighbors":[{"entityId":"1","distance":-0.99999988079071045,"entityKeyValues":{"keyValues":{"features":[{"name":"genre","value":{"stringValue":"Drama"}},{"name":"genre_code","value":{"int64Value":"5"}},{"name":"language","value":{"stringValue":"English"}},{"name":"running_time","value":{"int64Value":"142"}},{"name":"embedding","value":{"doubleArrayValue":{"values":[-0.055181887,0.037215218,0.007869319,0.034488678,-0.003985528,-0.002993597,0.02565795,0.022422727,0.009115592,-0.023928428,0.0076607596,-0.00036370804,0.001953341,0.04309559,0.027061913,-0.011557268,0.032698117,0.038843002,-0.027428165,0.02158849,0.014385543,-0.014721273,0.004527275,-0.028486224,0.029239075,0.02468128,-0.043950174,0.0046366416,0.014802663,-0.022137865,-0.020265914,-0.013185052,0.020520255,-0.0063178376,0.025454476,0.0019991226,0.002975793,-0.026593925,-0.023338357,-0.056239944,-0.071459725,-0.0061092777,-0.034895625,0.00051790243,0.009664969,0.028445529,-0.003141115,0.0012755216,0.00710

Search by entity


In [128]:
entity_id_to_find_neighbors_to = "2"
json_data = json.dumps(
    {"query": {"entity_id": entity_id_to_find_neighbors_to, "neighbor_count": 3}, "return_full_entity": True}
)

! curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d '{json_data}' \
    "https://1734071774.europe-west2-1012421088783.vdb.vertexai.goog/v1beta1/projects/208794590551/locations/europe-west2/featureOnlineStores/example_online_store/featureViews/movie_emb_featureview:searchNearestEntities"

{"nearestNeighbors":{"neighbors":[{"entityId":"2","distance":-0.99999994039535522,"entityKeyValues":{"keyValues":{"features":[{"name":"genre","value":{"stringValue":"Crime"}},{"name":"genre_code","value":{"int64Value":"4"}},{"name":"language","value":{"stringValue":"English"}},{"name":"running_time","value":{"int64Value":"175"}},{"name":"embedding","value":{"doubleArrayValue":{"values":[-0.077195555,0.06316,-0.025209697,-0.014495053,-0.026295781,0.0007244916,-0.014411508,-0.051756114,-0.022327395,-0.019392878,0.030326826,0.0286977,-0.010338691,-0.0025037385,0.006208436,0.01893338,0.009372702,0.030055305,-0.006067454,-0.030995186,0.014432395,0.04749532,-0.051463705,-0.029261628,-0.0043808897,0.004733345,-0.0004487279,0.025773626,0.04097881,-0.0073571797,0.014672587,0.0076234792,0.0066052745,-0.034441415,0.031120503,-0.044111747,0.028927447,-0.009430139,-0.042691484,0.002016828,-0.034900915,-0.018411225,-0.0013184441,-0.009148175,0.025543876,-0.013711819,0.035799023,-0.0075817066,0.02064